In [1]:
!pip install ibm-watsonx-ai==1.0.* 
!pip install autoai-libs~=2.0 
!pip install scikit-learn==1.3.*  
!pip install -U lale~=0.8.0  
!pip install xgboost==2.0.*  

ERROR: Ignored the following yanked versions: 1.14.2, 1.14.4, 2.0.0
ERROR: Ignored the following versions that require a different python version: 2.0.1 Requires-Python <3.12,>=3.11; 2.0.2 Requires-Python <3.12,>=3.11; 2.0.3 Requires-Python <3.12,>=3.11
ERROR: Could not find a version that satisfies the requirement autoai-libs~=2.0 (from versions: 1.14.0, 1.14.3, 1.14.6, 1.14.9, 1.14.10, 1.14.11, 1.14.12, 1.14.13, 1.15.0, 1.15.1, 1.15.2, 1.16.1, 1.16.2, 1.17.0, 1.17.2, 1.17.3)
ERROR: No matching distribution found for autoai-libs~=2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 68.6 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.1
    Uninstalling scikit-learn-1.1.1:
      Successfully uninstalled scikit-learn-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoai-libs 1.17.2 r

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
from ibm_watsonx_ai.helpers import DataConnection
from ibm_watsonx_ai.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='28c6ca7b-7551-4f79-9679-23f4d1a5695f'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/57cf47be-7752-448a-9d52-f64667e010eb/wml_data/c6344716-dd81-4e69-931d-abb33ffb2fc4/data/automl',
        model_location='auto_ml/57cf47be-7752-448a-9d52-f64667e010eb/wml_data/c6344716-dd81-4e69-931d-abb33ffb2fc4/data/automl/model.zip',
        training_status='auto_ml/57cf47be-7752-448a-9d52-f64667e010eb/wml_data/c6344716-dd81-4e69-931d-abb33ffb2fc4/training-status.json'
    )
)

In [4]:
experiment_metadata = dict(
    prediction_type='regression',
    prediction_column='SalePrice',
    holdout_size=0.1,
    scoring='neg_root_mean_squared_error',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=4,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    include_only_estimators=['RandomForestRegressorEstimator', 'DecisionTreeRegressorEstimator', 'RidgeEstimator', 'ExtraTreesRegressorEstimator', 'LinearRegressionEstimator', 'XGBRegressorEstimator', 'LGBMRegressorEstimator', 'SnapDecisionTreeRegressorEstimator', 'SnapRandomForestRegressorEstimator', 'SnapBoostingMachineRegressorEstimator', 'GradientBoostingRegressorEstimator'],
    deployment_url='https://eu-de.ml.cloud.ibm.com',
    project_id='b73c7089-0bbc-48de-96f2-adc27252935b',
    drop_duplicates=True,
    include_batched_ensemble_estimators=[],
    feature_selector_mode='auto'
)

In [5]:
import os, ast
CPU_NUMBER = 4
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

In [6]:
api_key = 'rf7wvhy4hsolItO8S3j4305EcR_9r6yBwJ7qpUOgzmBz'

In [7]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    api_key=api_key,
    url=experiment_metadata['deployment_url']
)

In [8]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

if 'space_id' in experiment_metadata:
    client.set.default_space(experiment_metadata['space_id'])
else:
    client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(client)

In [9]:
X_train, X_test, y_train, y_test = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=True)

In [10]:
from autoai_libs.transformers.exportable import ColumnSelector
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA2
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from xgboost import XGBRegressor

In [11]:
column_selector_0 = ColumnSelector(
    columns_indices_list=[
        1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
        55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79,
    ]
)
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[
        0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 37, 38, 39, 40, 43, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63,
        67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
    ]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "float_int_num", "char_str", "int_str", "char_str", "char_str",
        "char_str", "char_str", "char_str", "char_str", "char_str",
        "char_str", "char_str", "char_str", "char_str", "float_int_num",
        "float_int_num", "float_int_num", "char_str", "char_str", "char_str",
        "char_str", "char_str", "int_str", "char_str", "char_str", "char_str",
        "char_str", "char_str", "char_str", "char_str", "char_str",
        "char_str", "char_str", "char_str", "char_str", "float_int_num",
        "float_int_num", "float_int_num", "float_int_num", "float_int_num",
        "float_int_num", "float_int_num", "char_str", "float_int_num",
        "char_str", "float_int_num", "char_str", "char_str", "int_str",
        "char_str", "float_int_num", "char_str", "char_str", "char_str",
        "float_int_num", "float_int_num", "float_int_num", "char_str",
        "char_str", "char_str", "float_int_num", "float_int_num",
        "float_int_num", "char_str", "char_str",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [], [],
        ["NA"],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [],
        ["NA"],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [],
        ["NA"],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=["NA"]
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        100001, float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), 100001,
        100001, 100001, float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        100001, 100001, 100001, 100001, 100001, 100001, 100001, float("nan"),
        100001, float("nan"), 100001, float("nan"), float("nan"),
        float("nan"), float("nan"), 100001, float("nan"), float("nan"),
        float("nan"), 100001, 100001, 100001, float("nan"), float("nan"),
        float("nan"), 100001, 100001, 100001, float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
    encoding="ordinal",
    categories="auto",
)
pipeline_0 = make_pipeline(
    column_selector_0,
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
column_selector_1 = ColumnSelector(
    columns_indices_list=[
        1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
        55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79,
    ]
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[3, 17, 32, 34, 35, 36, 41, 42, 44, 60, 64, 65, 66]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "float_int_num", "float_int_num", "float_int_num", "float_int_num",
        "float_int_num", "float_int_num", "float_int_num", "float_int_num",
        "float_int_num", "float_int_num", "float_int_num", "float_int_num",
        "float_int_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    column_selector_1,
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 37, 38, 39, 40, 43, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63,
        67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 3, 17, 32, 34, 35, 36, 41,
        42, 44, 60, 64, 65, 66,
    ],
)
ta2_0 = TA2(
    fun=np.multiply,
    name="product",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "MSSubClass", "MSZoning", "LotFrontage", "LotArea", "Street", "Alley",
        "LotShape", "LandContour", "LotConfig", "LandSlope", "Neighborhood",
        "Condition1", "Condition2", "BldgType", "HouseStyle", "OverallQual",
        "OverallCond", "YearBuilt", "YearRemodAdd", "RoofStyle", "RoofMatl",
        "Exterior1st", "Exterior2nd", "MasVnrType", "MasVnrArea", "ExterQual",
        "ExterCond", "Foundation", "BsmtQual", "BsmtCond", "BsmtExposure",
        "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2",
        "BsmtUnfSF", "TotalBsmtSF", "Heating", "HeatingQC", "CentralAir",
        "Electrical", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",
        "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath",
        "BedroomAbvGr", "KitchenAbvGr", "KitchenQual", "TotRmsAbvGrd",
        "Functional", "Fireplaces", "FireplaceQu", "GarageType",
        "GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea",
        "GarageQual", "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF",
        "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "PoolQC",
        "Fence", "MiscFeature", "MiscVal", "MoSold", "YrSold", "SaleType",
        "SaleCondition",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 78),
    additional_col_count_to_keep=30,
    ptype="regression",
)
ta2_1 = TA2(
    fun=np.add,
    name="sum",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "MSSubClass", "MSZoning", "LotFrontage", "LotArea", "Street", "Alley",
        "LotShape", "LandContour", "LotConfig", "LandSlope", "Neighborhood",
        "Condition1", "Condition2", "BldgType", "HouseStyle", "OverallQual",
        "OverallCond", "YearBuilt", "YearRemodAdd", "RoofStyle", "RoofMatl",
        "Exterior1st", "Exterior2nd", "MasVnrType", "MasVnrArea", "ExterQual",
        "ExterCond", "Foundation", "BsmtQual", "BsmtCond", "BsmtExposure",
        "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2",
        "BsmtUnfSF", "TotalBsmtSF", "Heating", "HeatingQC", "CentralAir",
        "Electrical", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",
        "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath",
        "BedroomAbvGr", "KitchenAbvGr", "KitchenQual", "TotRmsAbvGrd",
        "Functional", "Fireplaces", "FireplaceQu", "GarageType",
        "GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea",
        "GarageQual", "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF",
        "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "PoolQC",
        "Fence", "MiscFeature", "MiscVal", "MoSold", "YrSold", "SaleType",
        "SaleCondition", "product(LotFrontage__YearRemodAdd)",
        "product(LotFrontage__GrLivArea)", "product(LotFrontage__GarageArea)",
        "product(LotArea__YearRemodAdd)", "product(YearBuilt__YearRemodAdd)",
        "product(YearBuilt__TotalBsmtSF)", "product(YearBuilt__1stFlrSF)",
        "product(YearBuilt__GrLivArea)", "product(YearBuilt__GarageArea)",
        "product(YearRemodAdd__BsmtFinSF1)",
        "product(YearRemodAdd__TotalBsmtSF)",
        "product(YearRemodAdd__1stFlrSF)", "product(YearRemodAdd__GrLivArea)",
        "product(YearRemodAdd__GarageYrBlt)",
        "product(YearRemodAdd__GarageArea)",
        "product(YearRemodAdd__WoodDeckSF)", "product(MasVnrArea__GrLivArea)",
        "product(MasVnrArea__GarageArea)", "product(TotalBsmtSF__1stFlrSF)",
        "product(TotalBsmtSF__GrLivArea)",
        "product(TotalBsmtSF__GarageYrBlt)",
        "product(TotalBsmtSF__GarageArea)", "product(1stFlrSF__GrLivArea)",
        "product(1stFlrSF__GarageYrBlt)", "product(1stFlrSF__GarageArea)",
        "product(GrLivArea__GarageYrBlt)", "product(GrLivArea__GarageArea)",
        "product(GarageYrBlt__GarageArea)", "product(GarageArea__WoodDeckSF)",
        "product(GarageArea__OpenPorchSF)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 78),
    additional_col_count_to_keep=30,
    ptype="regression",
)
xgb_regressor = XGBRegressor(
    gamma=1,
    learning_rate=0.02799548830927849,
    max_depth=4,
    min_child_weight=10,
    missing=float("nan"),
    n_estimators=313,
    n_jobs=CPU_NUMBER,
    random_state=33,
    reg_alpha=1,
    reg_lambda=0.24267724542081742,
    subsample=0.436914935459364,
    verbosity=0,
    silent=False,
)


In [12]:
pipeline = make_pipeline(
    union, numpy_permute_array, ta2_0, fs1_0, ta2_1, fs1_1, xgb_regressor
)

In [13]:
from sklearn.metrics import make_scorer

from autoai_libs.scorers.scorers import neg_root_mean_squared_error

scorer = make_scorer(neg_root_mean_squared_error)

In [14]:
pipeline.fit(X_train.values, y_train.values.ravel());

In [15]:
score = scorer(pipeline, X_test.values, y_test.values)
print(score)

-37800.012209087276


In [16]:
pipeline.predict(X_test.values[:5])

array([264746.78, 153484.98, 146863.06, 224411.33, 238531.34],
      dtype=float32)

In [17]:
model_metadata = {
    client.repository.ModelMetaNames.NAME: 'P4 - Pretrained AutoAI pipeline'
}

stored_model_details = client.repository.store_model(model=pipeline, meta_props=model_metadata, experiment_metadata=experiment_metadata)

In [18]:
stored_model_details

{'entity': {'hybrid_pipeline_software_specs': [{'id': '82f1bc6d-59c3-55e8-b612-cd908a2f7cc7',
    'name': 'autoai-kb_rt24.1-py3.11'}],
  'label_column': 'SalePrice',
  'pipeline': {'id': 'd12ae903-49ae-4941-a234-d3120125a287'},
  'schemas': {'input': [{'fields': [{'name': 'Id', 'type': 'double'},
      {'name': 'MSSubClass', 'type': 'double'},
      {'name': 'MSZoning', 'type': 'other'},
      {'name': 'LotFrontage', 'type': 'other'},
      {'name': 'LotArea', 'type': 'double'},
      {'name': 'Street', 'type': 'other'},
      {'name': 'Alley', 'type': 'other'},
      {'name': 'LotShape', 'type': 'other'},
      {'name': 'LandContour', 'type': 'other'},
      {'name': 'Utilities', 'type': 'other'},
      {'name': 'LotConfig', 'type': 'other'},
      {'name': 'LandSlope', 'type': 'other'},
      {'name': 'Neighborhood', 'type': 'other'},
      {'name': 'Condition1', 'type': 'other'},
      {'name': 'Condition2', 'type': 'other'},
      {'name': 'BldgType', 'type': 'other'},
      {'name

In [19]:
space_id = "4370294f-438e-45b1-84c1-67c315715a65"

model_id = client.spaces.promote(asset_id=stored_model_details["metadata"]["id"], source_project_id=experiment_metadata["project_id"], target_space_id=space_id)

In [20]:
client.set.default_space(space_id)

deploy_meta = {
        client.deployments.ConfigurationMetaNames.NAME: "Incrementally trained AutoAI pipeline",
        client.deployments.ConfigurationMetaNames.ONLINE: {},
    }

deployment_details = client.deployments.create(artifact_uid=model_id, meta_props=deploy_meta)
deployment_id = client.deployments.get_id(deployment_details)

Unsetting the project_id ...


######################################################################################

Synchronous deployment creation for id: '04c0d220-9cac-433e-a447-e15a2975b207' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
...
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='e9bda910-d6c5-43f4-ace8-fbb7bb1953ee'
-----------------------------------------------------------------------------------------------




In [21]:
import pandas as pd

scoring_payload = {
    "input_data": [{
        'values': pd.DataFrame(X_test[:5])
    }]
}

client.deployments.score(deployment_id, scoring_payload)

{'predictions': [{'fields': ['prediction'],
   'values': [[264746.78125],
    [153484.984375],
    [146863.0625],
    [224411.328125],
    [238531.34375]]}]}